In [1]:
import torch
import os


from script.plot_and_print import plot_data_scatter
from script.process_csv import generate_results, merge_data, process_spatial_data, ann_data_to_csv
import pandas as pd
from model import load_model

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
data_dir_train = "../Training_Data/"
data_dir_test = "../Test_Data/"
patients_train = [os.path.basename(f) for f in os.scandir(data_dir_train) if f.is_dir()]
patients_test = [os.path.basename(f) for f in os.scandir(data_dir_test) if f.is_dir()][1:]
print(patients_train)
print(patients_test)

model_dir = "../remote_models/"
skip = 0
model_dir_path = []

for model_type_dir in os.listdir(model_dir):
    sub_path = model_dir + model_type_dir
    if model_type_dir == ".DS_Store" or os.path.isfile(sub_path):
        continue
    for model_leaf_dir in os.listdir(sub_path):
        sub_path = model_dir + model_type_dir + "/" + model_leaf_dir
        if model_type_dir == ".DS_Store" or os.path.isfile(sub_path):
            continue
        files = os.listdir(sub_path)
        for f in files:
            if f[-3:] == ".pt" and f.find("ep_") != -1:
                src = sub_path + "/" + f
                dst = sub_path + "/" + f[f.find("ep_"):]
                os.rename(src, dst)
                if f[f.find("ep_"):] == "ep_29.pt":
                    model_dir_path.append((sub_path + "/", dst))
print(model_dir_path)
models = [load_model(x, y, log_json=True) for x, y in model_dir_path]

['p020', 'p016', 'p014', 'p013', 'p025', 'p007', 'p009']
['p026', 'p008']
[('../remote_models/vgg13/base_model/', '../remote_models/vgg13/base_model/ep_29.pt'), ('../remote_models/vgg13/dropout/', '../remote_models/vgg13/dropout/ep_29.pt'), ('../remote_models/res50/RUBCNL_Res50_freeze_tmp/', '../remote_models/res50/RUBCNL_Res50_freeze_tmp/ep_29.pt'), ('../remote_models/res50/RUBCNL_Res50_drop_freeze/', '../remote_models/res50/RUBCNL_Res50_drop_freeze/ep_29.pt'), ('../remote_models/res50/RUBCNL_Res50/', '../remote_models/res50/RUBCNL_Res50/ep_29.pt'), ('../remote_models/res18/RUBCNL_Res18_ciga_freeze_drop/', '../remote_models/res18/RUBCNL_Res18_ciga_freeze_drop/ep_29.pt'), ('../remote_models/res18/RUBCNL_Res18_freeze_drop/', '../remote_models/res18/RUBCNL_Res18_freeze_drop/ep_29.pt'), ('../remote_models/res18/RUBCNL_Res18_ciga_drop/', '../remote_models/res18/RUBCNL_Res18_ciga_drop/ep_29.pt'), ('../remote_models/res18/RUBCNL_Res18_ciga_freeze/', '../remote_models/res18/RUBCNL_Res18_ciga_

KeyError: 'gene_list'

In [3]:
for patient in patients:
    ann_data_to_csv(data_dir)
    process_spatial_data(patients, data_dir)
    merge_data(data_dir, patient)

In [16]:
def process_and_plot_patient(model, device, data_dir, patient, gene = "RUBCNL", results_filename = "results.csv"):
    generate_results(model, device, data_dir, patient, gene, results_filename)
    merge_data(data_dir, patient, results_filename)
    plot_data_scatter(data_dir, patient, gene)

In [10]:
columns = ["model_name", "model_type", "gene"]
to_append = [[s + "_mean_out", s + "_std_out", s + "_mean_labels", s + "_std_labels", s + "_pearson", s] for s in patients_test]
for s in to_append:
    for t in s:
        columns.append(t)
print(len(columns))
# mean_out, std_out, mean_labels, std_labels, pearson, patient
print(patients_test)

In [11]:
print(len(lines[0]))
df = pd.DataFrame(lines, columns=columns)
df.drop(patients_test, axis=1, inplace=True)
df.head()


In [12]:
df.to_csv("results_all_models_test.csv")

In [19]:
data_dir = "../Training_Data/"

patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patient = "p026"
for model in models:
    print(model[1])
    process_and_plot_patient(model[0].to(device), device, data_dir_test, patient, "RUBCNL", results_filename = "slim_model_results.csv")
    